In [ ]:
# !pip install langchain openai requests python-dotenv pandas

In [ ]:
import os
import json
import logging
import requests
from urllib.parse import urlparse
from dotenv import load_dotenv, find_dotenv

In [ ]:
load_dotenv(find_dotenv())

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [ ]:
GITHUB_ACCESS_TOKEN = os.environ.get("GITHUB_ACCESS_TOKEN")

In [ ]:
def is_empty(value): 
    if not value or value.strip() == "":
        return True
        
    return False

In [ ]:
def get_repo_from_url(github_url):
    parsed_url = urlparse(github_url)
    
    path_parts = parsed_url.path.strip('/').split('/')
    
    if len(path_parts) >= 2:
        owner = path_parts[0]
        repo = path_parts[1]
        return owner, repo
    else:
        print("Invalid GitHub URL")
        return None, None
        

In [ ]:
def fetch_github_data(endpoint, repo, params=None):
    if is_empty(GITHUB_ACCESS_TOKEN):
        logging.error("GITHUB_ACCESS_TOKEN is not set, is None, or is an empty string.")
        return False
      
    url = f"https://api.github.com/repos/{repo}/{endpoint}"

    headers = {
        "Authorization": f"Bearer {GITHUB_ACCESS_TOKEN}",
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28"
    }

    logging.info(f"Fetching data from {url} with params: {params}")
    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        logging.error(f"Failed to fetch data: HTTP {response.status_code}")
        logging.error(f"Response text: {response.text}")
        return None
    
    try:
        data = response.json()
        logging.info(f"Data fetched successfully from {url}")
        return data
    except requests.JSONDecodeError as e:
        logging.error("Failed to parse JSON response")
        logging.exception(e)
        return None
        

In [ ]:
def fetch_pull_request(repo, pull_request_url):
    pr_number = pull_request_url.split('/')[-1]
    endpoint = f"pulls/{pr_number}"
    return fetch_github_data(endpoint, repo)


In [ ]:
github_url = "https://github.com/apache/kafka"
owner, repo = get_repo_from_url(github_url)

print(f"Owner: {owner}\nRepository: {repo}")

In [ ]:
issues = fetch_github_data(endpoint="issues", repo=f"{owner}/{repo}", params={"state": "open"})
# print(json.dumps(issues, indent=4))

In [ ]:
max_iter_for_test = 2

if issues:
    for issue in issues:
        print(f"Issue #{issue['number']}: {issue['title']}")
        if 'pull_request' in issue:
            pr_data = fetch_pull_request("apache/kafka", issue['pull_request']['url'])
            # print(json.dumps(pr_data, indent=4))
        else:
            print("This issue is not a pull request.")

        max_iter_for_test -= 1
        if max_iter_for_test == 0:
            break


In [ ]:
import os
import datetime
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) 

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

llm_model = "gpt-3.5-turbo"

load_dotenv(find_dotenv()) 

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

llm_model = "gpt-3.5-turbo"

def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )

    print(response)
    return response.choices[0].message["content"]

# get_completion("What is 1+1?")